# 枠回しのアルゴリズム

In [3]:
import cvxpy
import numpy as np
import pandas as pd

[cvxpyの情報](https://www.cvxpy.org/tutorial/advanced/index.html)
<b></b>

```
Variable()     # スカラーを表す。
Variable(5)    # ベクトルを表す。
Variable(4, 7) # 行列を表す。
```

## ○線型計画法を用いる。
__<font color='Red'>花王のターゲットが主婦</font>、CCJCのターゲットがM3549であるが、<br>
花王の枠にM3549の情報がないため、<font color='Red'>CCJCのターゲットを女 20～34才</font>とする<br>
また、<font color='Red'>花王が号数6、CCJCが号数4（そのまま）</font>であるとする。__

In [68]:
# データの読み込み。
df_CCJC = pd.read_excel('sample/df_CCJC_sample.xlsx').loc[:, ["id","6_gou","世帯","女 20～34才","主婦"]].reset_index(drop=True)
df_kao = pd.read_excel('sample/df_kao_sample.xlsx').loc[:, ["id","6_gou","世帯","女 20～34才","主婦"]].reset_index(drop=True)

In [69]:
# 同じ向きに引くことがポイント！！
df_CCJC['相対価値'] = df_CCJC["女 20～34才"]-df_CCJC["主婦"]
df_kao['相対価値'] = df_kao["女 20～34才"]-df_kao["主婦"]

In [70]:
print("データ数:", len(df_kao))
df_kao.head(3)

データ数: 50


,id,6_gou,世帯,女 20～34才,主婦,相対価値
0,1,10.0,10.4,5.7,8.1,-2.4
1,2,6.6,6.1,4.1,5.4,-1.3
2,3,6.6,6.1,4.1,5.4,-1.3


In [71]:
print("データ数:", len(df_CCJC))
df_CCJC.head(3)

データ数: 50


,id,6_gou,世帯,女 20～34才,主婦,相対価値
0,1,6.6,8.7,3.9,6.5,-2.6
1,2,5.5,5.7,3.5,4.2,-0.7
2,3,5.5,5.7,3.5,4.2,-0.7


#### 相対価値のところで逆向きに引いた方（花王）の値を全てひっくり返す！

In [72]:
df_kao = df_kao.applymap(lambda x:-x)
df_kao.head(3)

,id,6_gou,世帯,女 20～34才,主婦,相対価値
0,-1,-10.0,-10.4,-5.7,-8.1,2.4
1,-2,-6.6,-6.1,-4.1,-5.4,1.3
2,-3,-6.6,-6.1,-4.1,-5.4,1.3


In [75]:
# そのままくっつける。
df = pd.concat([df_kao, df_CCJC]).reset_index(drop=True)

***
***

In [87]:
Pai = df['6_gou'].values # 花王の指定号数に従った枠の視聴率
Pbi = df['世帯'].values # CCJCの指定号数に従った枠の視聴率
Wi = df['相対価値'].values # 視聴率の上昇量(これを最大化したい)
Ci = cvxpy.Variable(len(Pai), boolean=True) # 取得するかどうかのフラグ
objective = cvxpy.Maximize(Wi * Ci) # 最大化したいもの（＝視聴率の上昇量）
constraints = [Pai * Ci == 0] + [Pbi * Ci == 0] # A,Bの号数が変わらないという制約条件
prob = cvxpy.Problem(objective, constraints) # 解く問題のこと。
prob.solve(solver=cvxpy.ECOS_BB) # solverを変える事で、解き方が変わる。
result = [round(ix, 0) for ix in Ci.value] # これで、フラグが0, 1で表される。

TypeError: 'NoneType' object is not iterable

In [ ]:
print("取得する枠かのフラグ:", result)

In [4]:
print("取得する枠の視聴率の上昇値:", np.array(result)*Wi)

取得する枠の視聴率の上昇値: [22.  0. 16. 10. 35.  0. 42. 53.]


In [5]:
print("上昇値の合計値:", sum(np.array(result)*Wi))

上昇値の合計値: 178.0
